In [ ]:
import sys
from pyspark import SparkContext, SparkConf

current_id = 0
def wordCountByID(alist):
    # get global variable
    global current_id
    # split each line by comma
    elements = alist[0].split(",")
    # remove double quotes
    idd = elements[0].replace('"', "")
    # if first element is a digit
    # it is an application id
    if(idd.isdigit()):
        # convert it to int
        current_id = int(idd)
        # return nothing
        return []
    # it is a list full of text
    else:
        # associate words with the current id
        return [[(current_id, (word, 1)) for word in statement.split()] for statement in elements]

# get ratings of top 10
def manipulate1(x):
    # column one is id
    # 1:-1 ignores the first and last quotations
    if int(x[1][1:-1]) in top10ids:
        # column 8 is overall rating
        return x[8]

# get ratings of least 10
def manipulate2(x):
    if int(x[1][1:-1]) in least10ids:
        return x[8]


if __name__ == '__main__':
    # create SparkContext with SparkConf
    conf = SparkConf().setAppName("Edureka_672184 - WordCount-TGA")
    sc = SparkContext(conf=conf)
    
    location = "/user/edureka_672184/use_cases/appleStore_description.csv"

    description = sc.textFile(location)
    
    pattern = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
    # split by pattern and ignore header
    description = description.map(lambda line: line.split(pattern))
    # remove white space
    description = description.filter(lambda cols: cols[0] != '')
    # remove header
    description = description.filter(lambda cols: 'id' not in cols[0])


    # get pairs like this (id, (word,1)    
    description_wList = description.map(wordCountByID).filter(lambda listt: len(listt) != 0)
    # flatten list of lists and group by key
    word_freq = description_wList.flatMap(lambda listt: listt[0]).groupByKey()
    # find length of each iteratable per group the sort it by that length in ascending order, then select top 10
    top10 = word_freq.map(lambda j: (j[0], len(list(j[1])))).sortBy(lambda line: line[1], ascending=False).take(10)
    # find least 10 
    least10 = word_freq.map(lambda j: (j[0], len(list(j[1])))).sortBy(lambda line: line[1], ascending=True).take(10)
    
    top10ids = [i[0] for i in top10]
    least10ids = [i[0] for i in least10]
    
    result1  = apple_lines.map(manipulate1).filter(lambda x: x).collect()
    result2  = apple_lines.map(manipulate2).filter(lambda x: x).collect()
    
    print("Showing ratings of highly described apps")
    for r in result1:
        print(r)
    print()
    print("Showing ratings of least described apps")
    for r in result2:
        print(r)
    print()